# Introduction
Jack Wilson
11/20/2025

This notebook outlines the analysis, modeling, and prediction of Formula 1 races

# Import Modules

In [1]:
import pandas as pd

# DataFrame Display Options

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)

# Pre-Qualifying

## Prep Data

### Import

In [26]:
pre_qual_ex = pd.read_csv('../data/final/f1_data_pre_qual_clean.csv')
pre_qual_ex.shape

(3335, 139)

### Remove Identifiers

In [27]:
pre_qual_ex.head(1)

,race_id,driver_id,circuit_id,team_id,year,round,driver_name,team_name,cumulative_races,cumulative_wins,cumulative_podiums,cumulative_points,avg_finish_last_3,avg_finish_last_5,avg_finish_last_10,position,points,laps_completed,CLAS_rate,NC_rate,DNF_rate,rookie_flag,circuit_name,type,direction,length,turns,elevation,yellow_flag_prob,double_yellow_prob,red_flag_prob,safety_car_prob,vsc_prob,avg_yellow_count,avg_double_yellow_count,avg_red_count,avg_safety_car_deployments,avg_vsc_deployments,avg_sc_laps,avg_vsc_laps,best_time_FP1,best_time_FP2,best_time_FP3,lap_count_FP1,lap_count_FP2,lap_count_FP3,position_FP1,position_FP2,position_FP3,recorded_lap_time_FP1,recorded_lap_time_FP2,recorded_lap_time_FP3,participated_FP1,participated_FP2,participated_FP3,avg_pit_time_last_5,avg_pit_time_last_10,AirTemp_mean,AirTemp_min,AirTemp_max,AirTemp_std,TrackTemp_mean,TrackTemp_min,TrackTemp_max,TrackTemp_std,WindSpeed_mean,WindSpeed_min,WindSpeed_max,WindSpeed_std,Humidity_mean,Humidity_min,Humidity_max,Humidity_std,Pressure_mean,Pressure_min,Pressure_max,Pressure_std,Rainfall_any,Rainfall_mean,used_soft_FP1,avg_pace_soft_FP1,std_pace_soft_FP1,deg_rate_soft_FP1,used_soft_FP2,avg_pace_soft_FP2,std_pace_soft_FP2,deg_rate_soft_FP2,used_soft_FP3,avg_pace_soft_FP3,std_pace_soft_FP3,deg_rate_soft_FP3,used_medium_FP1,avg_pace_medium_FP1,std_pace_medium_FP1,deg_rate_medium_FP1,used_medium_FP2,avg_pace_medium_FP2,std_pace_medium_FP2,deg_rate_medium_FP2,used_medium_FP3,avg_pace_medium_FP3,std_pace_medium_FP3,deg_rate_medium_FP3,used_hard_FP1,avg_pace_hard_FP1,std_pace_hard_FP1,deg_rate_hard_FP1,used_hard_FP2,avg_pace_hard_FP2,std_pace_hard_FP2,deg_rate_hard_FP2,used_hard_FP3,avg_pace_hard_FP3,std_pace_hard_FP3,deg_rate_hard_FP3,used_intermediate_FP1,avg_pace_intermediate_FP1,std_pace_intermediate_FP1,deg_rate_intermediate_FP1,used_intermediate_FP2,avg_pace_intermediate_FP2,std_pace_intermediate_FP2,deg_rate_intermediate_FP2,used_intermediate_FP3,avg_pace_intermediate_FP3,std_pace_intermediate_FP3,deg_rate_intermediate_FP3,used_wet_FP1,avg_pace_wet_FP1,std_pace_wet_FP1,deg_rate_wet_FP1,used_wet_FP2,avg_pace_wet_FP2,std_pace_wet_FP2,deg_rate_wet_FP2,used_wet_FP3,avg_pace_wet_FP3,std_pace_wet_FP3,deg_rate_wet_FP3
0,1,60,1,1,2018,1,Sebastian Vettel,Ferrari,198,47,99,2425.0,2.666667,5.8,5.7,1,25.0,58,0.855,0.14,0.0,False,Australia,Street circuit,Clockwise,5.278,14,8,0.904192,0.814371,0.11976,0.562874,0.413174,4.095808,3.538922,0.161677,0.784431,0.54491,3.065868,1.221557,84.995,84.451,86.067,22,38,15,5,5,1,True,True,True,True,True,True,24.1256,23.7166,19.772059,18.6,21.0,0.540549,24.758824,22.2,28.3,1.247713,0.754412,0.0,1.3,0.27287,86.838235,81.2,92.3,3.268257,1010.220588,1009.6,1011.5,0.523923,True,0.220588,True,91.193429,4.987276,-0.595135,True,94.16775,11.936055,-1.582934,True,86.067,0.0,0.0,True,85.911,0.0,0.0,True,89.304286,1.853079,-0.382151,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,True,95.912,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0


In [28]:
# Drop human-readable identifiers and data leakage columns
columns_to_drop = [
    'year',
    'round',
    'driver_name',
    'team_name',
    'points',
    'laps_completed',
    'circuit_name'
]
pre_qual_target = pre_qual_ex.drop(columns=columns_to_drop)

print(pre_qual_target.shape)
pre_qual_target.head(1)

(3335, 132)


,race_id,driver_id,circuit_id,team_id,cumulative_races,cumulative_wins,cumulative_podiums,cumulative_points,avg_finish_last_3,avg_finish_last_5,avg_finish_last_10,position,CLAS_rate,NC_rate,DNF_rate,rookie_flag,type,direction,length,turns,elevation,yellow_flag_prob,double_yellow_prob,red_flag_prob,safety_car_prob,vsc_prob,avg_yellow_count,avg_double_yellow_count,avg_red_count,avg_safety_car_deployments,avg_vsc_deployments,avg_sc_laps,avg_vsc_laps,best_time_FP1,best_time_FP2,best_time_FP3,lap_count_FP1,lap_count_FP2,lap_count_FP3,position_FP1,position_FP2,position_FP3,recorded_lap_time_FP1,recorded_lap_time_FP2,recorded_lap_time_FP3,participated_FP1,participated_FP2,participated_FP3,avg_pit_time_last_5,avg_pit_time_last_10,AirTemp_mean,AirTemp_min,AirTemp_max,AirTemp_std,TrackTemp_mean,TrackTemp_min,TrackTemp_max,TrackTemp_std,WindSpeed_mean,WindSpeed_min,WindSpeed_max,WindSpeed_std,Humidity_mean,Humidity_min,Humidity_max,Humidity_std,Pressure_mean,Pressure_min,Pressure_max,Pressure_std,Rainfall_any,Rainfall_mean,used_soft_FP1,avg_pace_soft_FP1,std_pace_soft_FP1,deg_rate_soft_FP1,used_soft_FP2,avg_pace_soft_FP2,std_pace_soft_FP2,deg_rate_soft_FP2,used_soft_FP3,avg_pace_soft_FP3,std_pace_soft_FP3,deg_rate_soft_FP3,used_medium_FP1,avg_pace_medium_FP1,std_pace_medium_FP1,deg_rate_medium_FP1,used_medium_FP2,avg_pace_medium_FP2,std_pace_medium_FP2,deg_rate_medium_FP2,used_medium_FP3,avg_pace_medium_FP3,std_pace_medium_FP3,deg_rate_medium_FP3,used_hard_FP1,avg_pace_hard_FP1,std_pace_hard_FP1,deg_rate_hard_FP1,used_hard_FP2,avg_pace_hard_FP2,std_pace_hard_FP2,deg_rate_hard_FP2,used_hard_FP3,avg_pace_hard_FP3,std_pace_hard_FP3,deg_rate_hard_FP3,used_intermediate_FP1,avg_pace_intermediate_FP1,std_pace_intermediate_FP1,deg_rate_intermediate_FP1,used_intermediate_FP2,avg_pace_intermediate_FP2,std_pace_intermediate_FP2,deg_rate_intermediate_FP2,used_intermediate_FP3,avg_pace_intermediate_FP3,std_pace_intermediate_FP3,deg_rate_intermediate_FP3,used_wet_FP1,avg_pace_wet_FP1,std_pace_wet_FP1,deg_rate_wet_FP1,used_wet_FP2,avg_pace_wet_FP2,std_pace_wet_FP2,deg_rate_wet_FP2,used_wet_FP3,avg_pace_wet_FP3,std_pace_wet_FP3,deg_rate_wet_FP3
0,1,60,1,1,198,47,99,2425.0,2.666667,5.8,5.7,1,0.855,0.14,0.0,False,Street circuit,Clockwise,5.278,14,8,0.904192,0.814371,0.11976,0.562874,0.413174,4.095808,3.538922,0.161677,0.784431,0.54491,3.065868,1.221557,84.995,84.451,86.067,22,38,15,5,5,1,True,True,True,True,True,True,24.1256,23.7166,19.772059,18.6,21.0,0.540549,24.758824,22.2,28.3,1.247713,0.754412,0.0,1.3,0.27287,86.838235,81.2,92.3,3.268257,1010.220588,1009.6,1011.5,0.523923,True,0.220588,True,91.193429,4.987276,-0.595135,True,94.16775,11.936055,-1.582934,True,86.067,0.0,0.0,True,85.911,0.0,0.0,True,89.304286,1.853079,-0.382151,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,True,95.912,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0,False,0.0,0.0,0.0


### One-Hot Encode

In [29]:
# One-hot encode 'type' and 'direction' columns
pre_qual_target = pd.get_dummies(pre_qual_target, columns=['type', 'direction'], prefix=['type', 'direction'])

### Convert Booleans

In [30]:
# Convert all boolean columns to integers
bool_columns = pre_qual_target.select_dtypes(include='bool').columns
for col in bool_columns:
    pre_qual_target[col] = pre_qual_target[col].astype(int)

In [31]:
pre_qual_target.head()

,race_id,driver_id,circuit_id,team_id,cumulative_races,cumulative_wins,cumulative_podiums,cumulative_points,avg_finish_last_3,avg_finish_last_5,avg_finish_last_10,position,CLAS_rate,NC_rate,DNF_rate,rookie_flag,length,turns,elevation,yellow_flag_prob,double_yellow_prob,red_flag_prob,safety_car_prob,vsc_prob,avg_yellow_count,avg_double_yellow_count,avg_red_count,avg_safety_car_deployments,avg_vsc_deployments,avg_sc_laps,avg_vsc_laps,best_time_FP1,best_time_FP2,best_time_FP3,lap_count_FP1,lap_count_FP2,lap_count_FP3,position_FP1,position_FP2,position_FP3,recorded_lap_time_FP1,recorded_lap_time_FP2,recorded_lap_time_FP3,participated_FP1,participated_FP2,participated_FP3,avg_pit_time_last_5,avg_pit_time_last_10,AirTemp_mean,AirTemp_min,AirTemp_max,AirTemp_std,TrackTemp_mean,TrackTemp_min,TrackTemp_max,TrackTemp_std,WindSpeed_mean,WindSpeed_min,WindSpeed_max,WindSpeed_std,Humidity_mean,Humidity_min,Humidity_max,Humidity_std,Pressure_mean,Pressure_min,Pressure_max,Pressure_std,Rainfall_any,Rainfall_mean,used_soft_FP1,avg_pace_soft_FP1,std_pace_soft_FP1,deg_rate_soft_FP1,used_soft_FP2,avg_pace_soft_FP2,std_pace_soft_FP2,deg_rate_soft_FP2,used_soft_FP3,avg_pace_soft_FP3,std_pace_soft_FP3,deg_rate_soft_FP3,used_medium_FP1,avg_pace_medium_FP1,std_pace_medium_FP1,deg_rate_medium_FP1,used_medium_FP2,avg_pace_medium_FP2,std_pace_medium_FP2,deg_rate_medium_FP2,used_medium_FP3,avg_pace_medium_FP3,std_pace_medium_FP3,deg_rate_medium_FP3,used_hard_FP1,avg_pace_hard_FP1,std_pace_hard_FP1,deg_rate_hard_FP1,used_hard_FP2,avg_pace_hard_FP2,std_pace_hard_FP2,deg_rate_hard_FP2,used_hard_FP3,avg_pace_hard_FP3,std_pace_hard_FP3,deg_rate_hard_FP3,used_intermediate_FP1,avg_pace_intermediate_FP1,std_pace_intermediate_FP1,deg_rate_intermediate_FP1,used_intermediate_FP2,avg_pace_intermediate_FP2,std_pace_intermediate_FP2,deg_rate_intermediate_FP2,used_intermediate_FP3,avg_pace_intermediate_FP3,std_pace_intermediate_FP3,deg_rate_intermediate_FP3,used_wet_FP1,avg_pace_wet_FP1,std_pace_wet_FP1,deg_rate_wet_FP1,used_wet_FP2,avg_pace_wet_FP2,std_pace_wet_FP2,deg_rate_wet_FP2,used_wet_FP3,avg_pace_wet_FP3,std_pace_wet_FP3,deg_rate_wet_FP3,type_Race circuit,type_Street circuit,direction_Anti-clockwise,direction_Clockwise,direction_Figure eight
0,1,60,1,1,198,47,99,2425.0,2.666667,5.8,5.7,1,0.855000,0.140000,0.0,0,5.278,14,8,0.904192,0.814371,0.11976,0.562874,0.413174,4.095808,3.538922,0.161677,0.784431,0.54491,3.065868,1.221557,84.995,84.451,86.067,22,38,15,5,5,1,1,1,1,1,1,1,24.1256,23.7166,19.772059,18.6,21.0,0.540549,24.758824,22.2,28.3,1.247713,0.754412,0.0,1.3,0.27287,86.838235,81.2,92.3,3.268257,1010.220588,1009.6,1011.5,0.523923,1,0.220588,1,91.193429,4.987276,-0.595135,1,94.167750,11.936055,-1.582934,1,86.067,0.0,0.0,1,85.9110,0.000000,0.000000,1,89.304286,1.853079,-0.382151,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1,95.912000,0.000000,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,1,0,1,0
1,1,57,1,22,207,62,117,2610.0,5.000000,3.4,2.6,2,0.885167,0.105263,0.0,0,5.278,14,8,0.904192,0.814371,0.11976,0.562874,0.413174,4.095808,3.538922,0.161677,0.784431,0.54491,3.065868,1.221557,84.026,83.931,94.225,27,35,7,1,1,8,1,1,1,1,1,1,23.7408,24.8133,19.772059,18.6,21.0,0.540549,24.758824,22.2,28.3,1.247713,0.754412,0.0,1.3,0.27287,86.838235,81.2,92.3,3.268257,1010.220588,1009.6,1011.5,0.523923,1,0.220588,1,93.152214,12.186640,-0.554316,1,94.567235,16.052836,-1.383569,0,0.000,0.0,0.0,0,0.0000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1,112.275333,15.360884,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,1,0,1,0
2,1,6,1,1,271,20,91,1565.0,3.333333,3.6,6.9,3,0.787546,0.208791,0.0,0,5.278,14,8,0.904192,0.814371,0.11976,0.562874,0.413174,4.095808,3.538922,0.161677,0.784431,0.54491,3.065868,1.221557,84.875,84.214,88.499,23,39,13,4,4,2,1,1,1,1,1,1,24.1078,24.9743,19.772059,18.6,21.0,0.540549,24.758824,22.2,28.3,1.247713,0.754412,0.0,1.3,0.27287

### Save

In [32]:
pre_qual_target.to_csv('../data/final/f1_data_pre_qual_final.csv', index=False)

## Analysis

### Load

In [2]:
pre_qual = pd.read_csv('../data/final/f1_data_pre_qual_final.csv')
pre_qual.head()

,race_id,driver_id,circuit_id,team_id,cumulative_races,cumulative_wins,cumulative_podiums,cumulative_points,avg_finish_last_3,avg_finish_last_5,...,deg_rate_wet_FP2,used_wet_FP3,avg_pace_wet_FP3,std_pace_wet_FP3,deg_rate_wet_FP3,type_Race circuit,type_Street circuit,direction_Anti-clockwise,direction_Clockwise,direction_Figure eight
0,1,60,1,1,198,47,99,2425.0,2.666667,5.8,...,0.0,0,0.0,0.0,0.0,0,1,0,1,0
1,1,57,1,22,207,62,117,2610.0,5.000000,3.4,...,0.0,0,0.0,0.0,0.0,0,1,0,1,0
2,1,6,1,1,271,20,91,1565.0,3.333333,3.6,...,0.0,0,0.0,0.0,0.0,0,1,0,1,0
3,1,79,1,14,128,5,28,816.0,15.333333,13.4,...,0.0,0,0.0,0.0,0.0,0,1,0,1,0
4,1,12,1,2,291,32,97,1849.0,9.000000,11.0,...,0.0,0,0.0,0.0,0.0,0,1,0,1,0


### Pipeline

In [12]:
# ============================================================================
# Formula 1 Finishing Position Prediction Pipeline
# ============================================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import spearmanr
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 1. LOAD AND PREPARE DATA
# ============================================================================

print("=" * 80)
print("STEP 1: DATA PREPARATION")
print("=" * 80)

# Assuming df is already loaded and cleaned
# df = pd.read_csv('your_data.csv')  # Uncomment if loading from file

# Store race_id for grouping before removal
race_groups = df['race_id'].copy()

# Remove ID columns
id_columns = ['race_id', 'driver_id', 'team_id']
existing_id_cols = [col for col in id_columns if col in df.columns]
df_clean = df.drop(columns=existing_id_cols)

print(f"✓ Removed ID columns: {existing_id_cols}")
print(f"✓ Dataset shape: {df_clean.shape}")

# Separate features and target
X = df_clean.drop(columns=['position'])
y = df_clean['position']

print(f"✓ Features: {X.shape[1]} columns")
print(f"✓ Target: position (range {y.min():.0f} to {y.max():.0f})")

# Split data using GroupShuffleSplit (no race leakage)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(X, y, groups=race_groups))

X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
race_val = race_groups.iloc[val_idx]

print(f"\n✓ Train set: {X_train.shape[0]} samples")
print(f"✓ Validation set: {X_val.shape[0]} samples")
print(f"✓ Unique races in validation: {race_val.nunique()}")

# ============================================================================
# 2. TRAIN REGRESSION MODELS
# ============================================================================

print("\n" + "=" * 80)
print("STEP 2: MODEL TRAINING")
print("=" * 80)

models = {}

# Model 1: Random Forest
print("\n[1/5] Training Random Forest Regressor...")
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
models['Random Forest'] = rf_model
print("✓ Random Forest trained")

# Model 2: Extra Trees
print("\n[2/5] Training Extra Trees Regressor...")
et_model = ExtraTreesRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)
et_model.fit(X_train, y_train)
models['Extra Trees'] = et_model
print("✓ Extra Trees trained")

# Model 3: XGBoost
print("\n[3/5] Training XGBoost Regressor...")
xgb_model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(X_train, y_train)
models['XGBoost'] = xgb_model
print("✓ XGBoost trained")

# Model 4: LightGBM
print("\n[4/5] Training LightGBM Regressor...")
lgb_model = lgb.LGBMRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)
lgb_model.fit(X_train, y_train)
models['LightGBM'] = lgb_model
print("✓ LightGBM trained")

# Model 5: CatBoost
print("\n[5/5] Training CatBoost Regressor...")
cb_model = CatBoostRegressor(
    iterations=100,
    depth=6,
    learning_rate=0.1,
    random_state=42,
    verbose=0
)
cb_model.fit(X_train, y_train)
models['CatBoost'] = cb_model
print("✓ CatBoost trained")

# ============================================================================
# 3. EVALUATE ALL MODELS
# ============================================================================

print("\n" + "=" * 80)
print("STEP 3: MODEL EVALUATION")
print("=" * 80)

def convert_predictions_to_ranks(predictions):
    """
    Convert regression predictions to unique race positions (1, 2, 3, ...).
    Lower predicted values get better (lower) positions.
    Handles ties by assigning unique ranks based on sort order.
    """
    # Get the indices that would sort the predictions
    sorted_indices = np.argsort(predictions)
    
    # Create rank array
    ranks = np.empty_like(sorted_indices)
    ranks[sorted_indices] = np.arange(1, len(predictions) + 1)
    
    return ranks

def calculate_race_spearman(y_true, y_pred, race_ids):
    """
    Calculate average Spearman correlation across all races.
    For each race, convert predictions to proper ranks (no duplicates),
    then compare to true finishing order.
    """
    correlations = []
    
    for race_id in race_ids.unique():
        mask = race_ids == race_id
        true_positions = y_true[mask].values
        pred_positions_raw = y_pred[mask]
        
        # Convert predictions to unique ranks
        pred_positions_ranked = convert_predictions_to_ranks(pred_positions_raw)
        
        # Skip races with only 1 driver (can't compute correlation)
        if len(true_positions) > 1:
            corr, _ = spearmanr(true_positions, pred_positions_ranked)
            correlations.append(corr)
    
    return np.mean(correlations)

# Store results
results = []

for model_name, model in models.items():
    print(f"\nEvaluating {model_name}...")
    
    # Make predictions
    y_pred = model.predict(X_val)
    
    # Calculate regression metrics
    mae = mean_absolute_error(y_val, y_pred)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    r2 = r2_score(y_val, y_pred)
    
    # Calculate race-level Spearman correlation
    avg_spearman = calculate_race_spearman(y_val, y_pred, race_val)
    
    results.append({
        'Model': model_name,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'Avg Spearman': avg_spearman
    })
    
    print(f"  MAE: {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R²: {r2:.4f}")
    print(f"  Avg Spearman: {avg_spearman:.4f}")

# ============================================================================
# 4. OUTPUT COMPARISON TABLE
# ============================================================================

print("\n" + "=" * 80)
print("STEP 4: MODEL COMPARISON")
print("=" * 80)

# Create results DataFrame and sort by MAE
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('MAE').reset_index(drop=True)

print("\n" + results_df.to_string(index=False))

# Identify best model
best_model_name = results_df.iloc[0]['Model']
best_model = models[best_model_name]

print(f"\n🏆 Best Model: {best_model_name} (lowest MAE)")

# ============================================================================
# 5. INFERENCE EXAMPLE (WITH PROPER RANKING)
# ============================================================================

print("\n" + "=" * 80)
print("STEP 5: INFERENCE EXAMPLE")
print("=" * 80)

# Example 1: Predict single driver position
print("\n--- Example 1: Single Driver Prediction ---")
print("Note: For a single driver, we can only show raw prediction.")
print("Proper ranking requires all drivers in the race.\n")

# Take a sample from validation set (without position)
sample_driver = X_val.iloc[0:1]
print(f"Input features shape: {sample_driver.shape}")

# Predict position
predicted_position_raw = best_model.predict(sample_driver)[0]

print(f"Predicted position (raw): {predicted_position_raw:.2f}")
print(f"⚠️  Without other drivers, we can't assign a unique position")

# Show actual position for comparison
actual_position = y_val.iloc[0]
print(f"Actual position: {actual_position}")

# Example 2: Predict full race grid order with PROPER RANKING
print("\n--- Example 2: Full Race Grid Prediction (WITH RANKING) ---")

# Get all drivers from first race in validation set
first_race_id = race_val.iloc[0]
race_mask = race_val == first_race_id
race_data = X_val[race_mask]
race_actual = y_val[race_mask]

# Predict positions for all drivers in the race
race_predictions_raw = best_model.predict(race_data)

# ⭐ CONVERT TO PROPER RANKS (no duplicates!)
race_predictions_ranked = convert_predictions_to_ranks(race_predictions_raw)

# Create race results DataFrame
race_results = pd.DataFrame({
    'Driver_Index': race_data.index,
    'Predicted_Position_Raw': race_predictions_raw,
    'Predicted_Position_Ranked': race_predictions_ranked.astype(int),
    'Actual_Position': race_actual.values
})

# Sort by raw predictions to show predicted grid order
race_results_sorted = race_results.sort_values('Predicted_Position_Raw').reset_index(drop=True)
race_results_sorted['Predicted_Grid_Order'] = range(1, len(race_results_sorted) + 1)

print(f"\nRace ID: {first_race_id}")
print(f"Number of drivers: {len(race_results_sorted)}")
print("\n✅ Predicted Grid Order (with proper unique rankings):")
print(race_results_sorted[['Predicted_Grid_Order', 'Driver_Index', 'Predicted_Position_Raw', 'Predicted_Position_Ranked', 'Actual_Position']].head(10).to_string(index=False))

if len(race_results_sorted) > 10:
    print("... (showing top 10)")

# Calculate Spearman correlation for this race
race_spearman, _ = spearmanr(race_results['Actual_Position'], race_results['Predicted_Position_Ranked'])
print(f"\n📊 Spearman correlation for this race: {race_spearman:.4f}")

# Show improvement with ranking
print("\n" + "─" * 80)
print("💡 KEY INSIGHT: Ranking eliminates duplicates!")
print("─" * 80)
print("Before ranking: Predictions might be [5.2, 5.3, 5.1, 5.2] → duplicates!")
print("After ranking:  Predictions become [2, 4, 1, 3] → unique positions ✓")
print("\nThis is crucial for real-world F1 predictions where each position is unique.")

# ============================================================================
# SAVE BEST MODEL (OPTIONAL)
# ============================================================================

print("\n" + "=" * 80)
print("PIPELINE COMPLETE")
print("=" * 80)

print(f"\n✓ Best model ({best_model_name}) is stored in variable: best_model")
print("✓ To use the model: predictions = best_model.predict(X_new)")
print("✓ To save the model: import joblib; joblib.dump(best_model, 'f1_model.pkl')")

# Optional: Save the best model
# import joblib
# joblib.dump(best_model, 'f1_best_model.pkl')
# print("✓ Model saved to 'f1_best_model.pkl'")

STEP 1: DATA PREPARATION
✓ Removed ID columns: ['race_id', 'driver_id', 'team_id']
✓ Dataset shape: (3335, 132)
✓ Features: 131 columns
✓ Target: position (range 1 to 20)

✓ Train set: 2656 samples
✓ Validation set: 679 samples
✓ Unique races in validation: 34

STEP 2: MODEL TRAINING

[1/5] Training Random Forest Regressor...
✓ Random Forest trained

[2/5] Training Extra Trees Regressor...
✓ Extra Trees trained

[3/5] Training XGBoost Regressor...
✓ XGBoost trained

[4/5] Training LightGBM Regressor...
✓ LightGBM trained

[5/5] Training CatBoost Regressor...
✓ CatBoost trained

STEP 3: MODEL EVALUATION

Evaluating Random Forest...
  MAE: 3.6170
  RMSE: 4.5291
  R²: 0.3798
  Avg Spearman: 0.6121

Evaluating Extra Trees...
  MAE: 3.6297
  RMSE: 4.5304
  R²: 0.3794
  Avg Spearman: 0.6084

Evaluating XGBoost...
  MAE: 3.7714
  RMSE: 4.7385
  R²: 0.3211
  Avg Spearman: 0.5804

Evaluating LightGBM...
  MAE: 3.6699
  RMSE: 4.6697
  R²: 0.3407
  Avg Spearman: 0.5919

Evaluating CatBoost...
  M

# Pre-Race